<a href="https://colab.research.google.com/github/Somabss1/AIML/blob/main/GoogleGemini/Ollama_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langchain-openai pypdf chromadb transformers

In [ ]:
# Loading required libraries
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain import HuggingFaceHub
from langchain.llms import Ollama

First we will create our vector database and add our documents, you can use any but I'm using chroma.

Since I'm a big fan of Elden Ring, lets chat with Ollama about that game.

In [ ]:
data_path = "./data/Elden_Ring.pdf"

First we define the text splitter and add our pdf to the vector Database, I'll use OpenAI Embeddings, but you can use any embedding model.

In [ ]:
embedding_func = OpenAIEmbeddings(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=30,
    length_function=len,)

In [ ]:
documents = PyPDFLoader(data_path).load_and_split(text_splitter=text_splitter)

In [ ]:
print(documents[2].page_content)

Shoi Miyazawa
Tai Tomisawa
Yuka Kitamura
Yoshimi Kudo
P l a t f o r m ( s ) PlayStation 4
PlayStation 5
Windows
Xbox One
Xbox Series X/S
R e l e a s e February 25,
2022
G e n r e ( s ) Action role-
playing
M o d e ( s ) Single-player,
multiplayer
The player character fighting a
dragon while riding Torrent, their
steedsix main areas, which
include Limgrave—an area
of grassy plains and ancient
ruins—and Caelid, a
wasteland that is home to
unde ad monsters.[1] The
open world is explored
using the character's mount
Torrent as the main mode
of transportation, though
players may use fast travel
outside combat.
Throughout  the game, players encounter non-player characters
(NPCs) and enemies, including demigods  who rule each main area
and serve as the game's main bosses.[2][3] Aside from the main
areas, Elden Ring has hidden dunge ons, catacombs, tunne ls, and
caves where players can fight bosses and ga ther helpful items.[4]
At the game's start, the player choos es a character class, which
d

In [ ]:
vectordb = Chroma.from_documents(documents, embedding=embedding_func)

Now we define a prompt template

In [ ]:
template = """<s>[INST] Given the context - {context} </s>[INST] [INST] Answer the following question - {question}[/INST]"""
pt = PromptTemplate(
            template=template, input_variables=["context", "question"]
        )

Now we are all set to create our RAG, all we need to do is create a memeory and pass the vectordb as a retriever

You should have Ollama running, to set up ollama you can go to https://ollama.com/. Its pretty simple.

In [ ]:
rag = RetrievalQA.from_chain_type(
            llm=Ollama(model="mistral"),
            retriever=vectordb.as_retriever(),
            memory=ConversationSummaryMemory(llm = Ollama(model="mistral")),
            chain_type_kwargs={"prompt": pt, "verbose": True},
        )

In [ ]:
rag.invoke("What is Elden Ring ?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Given the context - Shoi Miyazawa
Tai Tomisawa
Yuka Kitamura
Yoshimi Kudo
P l a t f o r m ( s ) PlayStation 4
PlayStation 5
Windows
Xbox One
Xbox Series X/S
R e l e a s e February 25,
2022
G e n r e ( s ) Action role-
playing
M o d e ( s ) Single-player,
multiplayer
The player character fighting a
dragon while riding Torrent, their
steedsix main areas, which
include Limgrave—an area
of grassy plains and ancient
ruins—and Caelid, a
wasteland that is home to
unde ad monsters.[1] The
open world is explored
using the character's mount
Torrent as the main mode
of transportation, though
players may use fast travel
outside combat.
Throughout  the game, players encounter non-player characters
(NPCs) and enemies, including demigods  who rule each main area
and serve as the game's main bosses.[2][3] Aside from the main
areas, Elden Ring has hidden dunge ons, catacombs, tunne ls, an

{'query': 'What is Elden Ring ?',
 'history': '',
 'result': ' Elden Ring is a 2022 action role-playing game developed by FromSoftware. It was published for PlayStation 4, PlayStation 5, Windows, Xbox One, and Xbox Series X/S. In the game, players control a customizable character on a quest to repair the Elden Ring and become the new Elden Lord. The game is set in an open world, presented through a third-person perspective, and includes several types of weapons and magic spells. Players can traverse the six main areas using their steed Torrent and discover linear hidden dungeons and checkpoints that enable fast travel and attribute improvements. Elden Ring features online multiplayer mode for cooperative play or player-versus-player combat. The game was developed with inspirations from Dark Souls series, and contributions from George R.R. Martin on the narrative and Tsukasa Saitoh, Shoi Miyazawa, Tai Tomisawa, Yuka Kitamura, and Yoshimi Kudo for the original soundtrack. Elden Ring rece

In [ ]:
rag.invoke("Which awards did Elden Ring Win ?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Given the context - Elden Ring winning Game of the Year
at the 23rd Game Developers Choice
AwardsSome reviewers criticized a number of the game's menu and accessibility systems.[84][85] Reviewers
complained about the poor performance of the Window s version; framerate issues were commonly
mentioned.[81][86] Reviewers noted the story of Elden Ring lacks Martin's writing style. Kyle Orland of Ars
Technica said the game's storytelling is "characteristically sparse and cryptic", and differs from the
expectations of Martin's fans.[76] Chris Carter of Destructoid called the story "low key" but said it is better-
told than those of previous FromSoftware games.[80] Aoife Wilson of Eurogam er said George R. R.
Martin's heavy inclusion in the marketing was "baffling" when his contributions to the overall narrative
were unclear.[72] Mitchell Saltzman did not mind the lack of Martin'